In [2]:
using NamedArrays
n = NamedArray(rand(2,6))

setnames!(n, ["Celtics", "Bucks"], 1)         # give the names "one" and "two" to the rows (dimension 1)
n["Celtics", 2:3]
n["Bucks", :] = 11:16
n[Not("Bucks"), :] = 4:9                      # all rows but the one called "two"

@show n;

n = 2×6 Named Array{Float64,2}
  A ╲ B │    1     2     3     4     5     6
────────┼───────────────────────────────────
Celtics │  4.0   5.0   6.0   7.0   8.0   9.0
Bucks   │ 11.0  12.0  13.0  14.0  15.0  16.0


In [56]:
using CSV, NamedArrays
L = CSV.read("FBStats.csv")
println(L)

627×7 DataFrames.DataFrame. Omitted printing of 3 columns
│ Row │ Starter (=1) │ FULL NAME        │ TEAM    │ POS     │
├─────┼──────────────┼──────────────────┼─────────┼─────────┤
│ 1   │ missing      │ missing          │ missing │ missing │
│ 2   │ 1            │ Dewayne Dedmon   │ Atl     │ C       │
│ 3   │ missing      │ Isaac Humphries  │ Atl     │ C       │
│ 4   │ missing      │ Alex Len         │ Atl     │ C       │
│ 5   │ missing      │ Miles Plumlee    │ Atl     │ C       │
│ 6   │ missing      │ Tyler Zeller     │ Atl     │ C       │
│ 7   │ missing      │ Deyonta Davis    │ Atl     │ C       │
│ 8   │ missing      │ DeAndre' Bembry  │ Atl     │ F       │
⋮
│ 619 │ missing      │ Ron Baker        │ Was     │ G       │
│ 620 │ 1            │ Bradley Beal     │ Was     │ G       │
│ 621 │ missing      │ Jordan McRae     │ Was     │ G       │
│ 622 │ missing      │ Gary Payton II   │ Was     │ G       │
│ 623 │ missing      │ Chasson Randle   │ Was     │ G       │
│ 624 │ mi

In [57]:
function fCenter(i)
    if (length(c) == 0)
        push!(c, i)
    elseif ((L[c[1],5] - L[c[1], 6]) < (L[i, 5] - L[i, 6]))
        c[1] = i
    end
end

function fForward(i)
    if (length(f) == 0)
        push!(f, i)
    elseif ((L[f[1],5] - L[f[1], 6]) <= (L[i, 5] - L[i, 6]))
        if length(f) != 1
            pop!(f)
        end
        unshift!(f, i)
    elseif (length(f) == 1)
        push!(f, i)
    elseif ((L[f[2],5] - L[f[2], 6]) < (L[i, 5] - L[i, 6]))
        pop!(f)
        push!(f, i)
    end
end

function fGuard(i)
    if (length(g) == 0)
        push!(g, i)
    elseif ((L[g[1],5] - L[g[1], 6]) <= (L[i, 5] - L[i, 6]))
        if length(g) != 1
            pop!(g)
        end
        unshift!(g, i)
    elseif (length(g) == 1)
        push!(g, i)
    elseif ((L[g[2],5] - L[g[2], 6]) < (L[i, 5] - L[i, 6]))
        pop!(g)
        push!(g, i)
    end
end

c = Int64[]
f = Int64[]
g = Int64[]
team = 1
bench = Array{String}(30,5)
for i in 2:627
    if (typeof(L[i, 7]) == Missings.Missing)
        if (typeof(L[i,1]) == Missings.Missing)
            if (L[i,4][1] == 'C')
                fCenter(i)
                continue
            end
            if (L[i,4][1] == 'G')
                fGuard(i)
                continue
            end
            if (length(L[i,4]) == 3)
                if (L[i,4][3] == 'C')
                    fCenter(i)
                    continue
                end
                if (L[i,4][3] == 'F')
                    fForward(i)
                    continue
                end
                if (L[i,4][3] == 'G')
                    fGuard(i)
                    continue
                end
            end
            if (L[i,4][1] == 'F')
                fForward(i)
                continue
            end
        end
    else
        bench[team, 1] = L[c[1], 2]
        bench[team, 2] = L[f[1], 2]
        bench[team, 3] = L[f[2], 2]
        bench[team, 4] = L[g[1], 2]
        bench[team, 5] = L[g[2], 2]
        team = team + 1
        c = Int64[]
        f = Int64[]
        g = Int64[]
    end
end

In [58]:
a = NamedArray(bench)
show(a[24,:]) #24

5-element Named Array{String,1}
B  │ 
───┼─────────────────
1  │ "Richaun Holmes"
2  │  "Emanuel Terry"
3  │   "Ray Spalding"
4  │  "Tyler Johnson"
5  │   "Troy Daniels"

In [305]:
D = CSV.read("TrueDataSet.csv")
println(D)
S = CSV.read("Salary.csv")
println(S)

331×7 DataFrames.DataFrame. Omitted printing of 5 columns
│ Row │ Starter (=1) │ FULL NAME        │
├─────┼──────────────┼──────────────────┤
│ 1   │ missing      │ missing          │
│ 2   │ 1            │ Dewayne Dedmon   │
│ 3   │ 1            │ Taurean Prince   │
│ 4   │ 1            │ John Collins     │
│ 5   │ 1            │ Kent Bazemore    │
│ 6   │ 1            │ Trae Young       │
│ 7   │ missing      │ Deyonta Davis    │
│ 8   │ missing      │ BJ Johnson       │
⋮
│ 323 │ 1            │ Otto Porter Jr.  │
│ 324 │ 1            │ Bradley Beal     │
│ 325 │ 1            │ John Wall        │
│ 326 │ missing      │ Thomas Bryant    │
│ 327 │ missing      │ Jeff Green       │
│ 328 │ missing      │ Devin Robinson   │
│ 329 │ missing      │ Gary Payton II   │
│ 330 │ missing      │ Tomas Satoransky │
│ 331 │ missing      │ missing          │
601×4 DataFrames.DataFrame
│ Row │ Rk  │ Player             │           │ 2018-19   │
├─────┼─────┼────────────────────┼───────────┼──────────

In [289]:
d = Array{Union{Float64}, 2}(30, 5)
n = 0
for i in 1:30
    d[i,1] = sum(D[2+n:6+n,3])
    d[i,2] = sum(D[7+n:11+n,3])

    d[i,3] = sum(D[2+n:6+n,4])
    d[i,4] = sum(D[7+n:11+n,4])
    n = n + 11
end

d[:,5] = 1 # Set constant for each team (W_0)

nd = NamedArray(d)
setnames!(nd, ["Atlanta", "Boston", "Brooklyn", "Cha", "Chi", "Clevland", "Dallas", "Denver", "Detroit", "Gol", "Houston", 
        "Ind", "LaC", "LaL", "Memphis", "Miami", "Milwaukee", "Minnesota", "Nor", "Nyk", "Okc", "Orlando", "Phi", "Pho", "Por", "Sac", "San", "Tor", "Uta", "Was"], 1)
setnames!(nd, ["Starter OffR", "Bench OffR", "Starter DefR", "Bench DefR", "Constant"], 2)
show(nd)

30×5 Named Array{Float64,2}
    A ╲ B │ Starter OffR    Bench OffR  …    Bench DefR      Constant
──────────┼──────────────────────────────────────────────────────────
Atlanta   │        549.7         588.6  …         536.0           1.0
Boston    │        566.2         630.5            488.7           1.0
Brooklyn  │        540.1         611.5            493.1           1.0
Cha       │        574.8         567.6            527.2           1.0
Chi       │        526.9         570.8            529.3           1.0
Clevland  │        550.8         577.0            537.6           1.0
Dallas    │        539.9         607.9            528.8           1.0
Denver    │        555.6         591.7            517.8           1.0
Detroit   │        543.2         603.1            478.8           1.0
⋮                      ⋮             ⋮  ⋱             ⋮             ⋮
Orlando   │        556.7         608.6            492.5           1.0
Phi       │        528.2         635.8            513.0       

In [306]:
pRost = Array{Union{String, Float64}, 2}(300, 4)
n = 1
for nl in 1:11
    for i in nl:11:330
        if (typeof(D[i, 2]) != Missings.Missing)
            for j in 1:3
                pRost[n, j] = D[i, j + 1]
            end
            n += 1
        end
    end
end
for i in 1:601
    for j in 1:300
        if (S[i, 2] == pRost[j, 1])
            pRost[j, 4] = S[i, 4]
        end
    end
end

In [316]:
#println(pRost)
pn = NamedArray(pRost)
setnames!(pn, ["Player", "OffR", "DefR", "Salary"], 2)
show(pn)

300×4 Named Array{Union{Float64, String},2}
A ╲ B │             Player  …              Salary
──────┼──────────────────────────────────────────
1     │   "Dewayne Dedmon"  …               7.2e6
2     │       "Al Horford"              2.89287e7
3     │    "Jarrett Allen"              2.03412e6
4     │      "Cody Zeller"              1.35281e7
5     │      "Robin Lopez"              1.43578e7
6     │       "Kevin Love"               2.4119e7
7     │   "DeAndre Jordan"                 2.29e7
8     │     "Nikola Jokic"              2.46052e7
9     │   "Andre Drummond"              2.54343e7
⋮                        ⋮  ⋱                   ⋮
292   │     "Jerian Grant"              2.63931e6
293   │    "Landry Shamet"              1.70364e6
294   │    "Tyler Johnson"              1.92454e7
295   │      "Rodney Hood"              3.47289e6
296   │     "Corey Brewer"               270496.0
297   │      "Patty Mills"              1.15714e7
298   │      "Jodie Meeks"               122953.0
299   

In [ ]:
#Jalen Jones, Marcus Derrickson, Isaiah Hicks, Amile Jefferson, Alex Poythress, Johnathan Motley, Johnathan Williams, Devin Robinson
#RJ Hunter, alex caruso, kadeem allen, Jordan Loyd, kalin lucas